In [1]:
import pandas as pd
import os
import numpy as np

import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF

#from modAL.models import ActiveLearner

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
import math

%matplotlib inline

In [2]:
# cleaned version of the data after removing outliers and negative values
path='/workspace/malathi/steam_turbine_stage_1_POC/notebooks/version_1/df_sorted.csv'
data=pd.read_csv(path)
data.shape

(7077, 6)

In [3]:
INPUT_COLUMNS=[ 'expansion ratio','corrected speed','ImpulseStage_CurrentStroke']
OUTPUT_COLUMNS=['ImpulseStage_Power', 'Impulse_Discharge_Temperature', 'corrected mass flow']

In [4]:
# predictions for 'Impulsestage_Power'
X=data.drop(OUTPUT_COLUMNS,axis=1)
y=data['ImpulseStage_Power']
print(X.shape,y.shape)


(7077, 3) (7077,)


In [6]:
# XX=X.values.reshape(-1,3)
# yy=y.values.reshape(-1,1)
#print(XX.shape,yy.shape)

In [7]:
X_pool, X_test, y_pool, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
#X_pool, X_test, y_pool, y_test = X_pool.reset_index(drop=True), X_test.reset_index(drop=True), y_pool.reset_index(drop=True), y_test.reset_index(drop=True)
# random state 1 5 iterations
# random state 2 20 iteration

In [8]:
X_pool_1=X_pool.copy()
X_test_1=X_test.copy() 
y_pool_1=y_pool.copy()
y_test_1=y_test.copy()

In [34]:
X_pool, X_test, y_pool, y_test = X_pool.reset_index(drop=True), X_test.reset_index(drop=True), \
y_pool.reset_index(drop=True), y_test.reset_index(drop=True)

In [35]:
# take 100 points from the pool, train them
train_idx=list(range(100))
# keep extending this range to get more ambiguous points
unknown_idx=list(range(100,2000))

In [36]:
X_train=X_pool.iloc[train_idx]
y_train=y_pool.iloc[train_idx]

In [37]:
X_unk = X_pool.iloc[unknown_idx]

In [38]:
estimator=GaussianProcessRegressor()
model=estimator.fit(X_train,y_train)

In [39]:
def find_most_ambiguous(model,data):
    _, std = model.predict(data, return_std=True)
    query_idx = np.argmax(std)
    print(query_idx)
    return query_idx, X_pool.iloc[query_idx]

In [40]:
# find ambiguous points
idx,ambiguous_data=find_most_ambiguous(model,X_unk)

56


/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/_gpr.py:362: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [41]:
X_pool.iloc[56]

expansion ratio                 7.427656
corrected speed               143.980509
ImpulseStage_CurrentStroke     10.590000
Name: 56, dtype: float64

In [42]:
ambiguous_data

expansion ratio                 7.427656
corrected speed               143.980509
ImpulseStage_CurrentStroke     10.590000
Name: 56, dtype: float64

## Option 1

### For these ambiguous points find out the labels throgh simulation using matlab code
### Or through Reinforcement learning

In [55]:
# for range(100,1000) in x_pool unambiguous point is 56

In [18]:
#X_unk1=X_unk.reset_index(drop=True)
# dont do this Gives different results

# Option    2
#### Include these points in the training set and again train them to get the best results

In [111]:
print(len(X_train),len(y_train))

100 100


In [112]:
xx=X_train.copy()
yy=y_train.copy()

In [113]:
xx=xx.append(X_pool.iloc[56])
yy=yy.append(pd.Series(y_pool.iloc[56]))
#yy.append(y_pool.iloc[56])

In [114]:
xx=xx.reset_index(drop=True)
yy=yy.reset_index(drop=True)

In [115]:
X_pool.drop(X_pool.index[56],inplace=True)
y_pool.drop(X_pool.index[56],inplace=True)

X_pool=X_pool.reset_index(drop=True)
y_pool=y_pool.reset_index(drop=True)

In [116]:
X_pool.tail()

,expansion ratio,corrected speed,ImpulseStage_CurrentStroke
5654,4.432234,133.215052,16.31
5655,1.631742,135.091320,36.35
5656,2.116215,134.163020,30.62
5657,7.264670,119.298136,10.59
5658,1.621527,141.738218,36.35


In [117]:
print(X_pool.shape,y_pool.shape)

(5659, 3) (5659,)


In [ ]:
# train them again

In [118]:
len(xx)

101

In [119]:
# take 101 points from the pool, train them
train_idx=list(range(102))
# keep extending this range to get more ambiguous points
unknown_idx=list(range(102,2000))

In [120]:
# X_train=X_pool.iloc[train_idx]
# y_train=y_pool.iloc[train_idx]

X_train_new=xx.copy()
y_train_new=yy.copy()

X_unk = X_pool.iloc[unknown_idx]

estimator=GaussianProcessRegressor()
model=estimator.fit(X_train_new,y_train_new)

def find_most_ambiguous(model,data):
    _, std = model.predict(data, return_std=True)
    query_idx = np.argmax(std)
    print(query_idx)
    return query_idx, X_pool.iloc[query_idx]

# find ambiguous points
idx,ambiguous_data=find_most_ambiguous(model,X_unk)

52


/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/_gpr.py:362: UserWarning: Predicted variances smaller than 0. Setting those variances to 0.
  warnings.warn("Predicted variances smaller than 0. "


In [124]:
X_pool.iloc[52]

expansion ratio                 4.427210
corrected speed               132.495649
ImpulseStage_CurrentStroke     16.310000
Name: 52, dtype: float64

In [125]:
y_pool.iloc[52]

713.5114746

In [126]:
X_train_new=X_train_new.append(X_pool.iloc[52])
y_train_new=y_train_new.append(pd.Series(y_pool.iloc[52]))



X_train_new=X_train_new.reset_index(drop=True)
y_train_new=y_train_new.reset_index(drop=True)
print(len(X_train_new),len(y_train_new))

X_pool.drop(X_pool.index[52],inplace=True)
y_pool.drop(X_pool.index[52],inplace=True)

X_pool=X_pool.reset_index(drop=True)
y_pool=y_pool.reset_index(drop=True)


print(len(X_pool),len(y_pool))

103 103
5658 5658


In [132]:
# train them again

# take 101 points from the pool, train them
train_idx=list(range(104))
# keep extending this range to get more ambiguous points
unknown_idx=list(range(5600,5657))
X_unk = X_pool.iloc[unknown_idx]

estimator=GaussianProcessRegressor()
model=estimator.fit(X_train_new,y_train_new)

def find_most_ambiguous(model,data):
    _, std = model.predict(data, return_std=True)
    #print(std)
    
    query_idx = np.argmax(std)
    print(max(std))
    
    print(query_idx)
    return query_idx, X_pool.iloc[query_idx]

# find ambiguous points
idx,ambiguous_data=find_most_ambiguous(model,X_unk)

0.9999999999462106
44


## Ignore these cells 

In [22]:
X_train=X_pool.iloc[56]

In [24]:
X_train.head(5)

expansion ratio                 7.427656
corrected speed               143.980509
ImpulseStage_CurrentStroke     10.590000
Name: 56, dtype: float64

In [25]:
# Creating the first Dataframe using dictionary
df1 = df = pd.DataFrame({"a":[1, 2, 3, 4],
                         "b":[5, 6, 7, 8]})
  
# Creating the Second Dataframe using dictionary
df2 = pd.DataFrame({"a":[1, 2, 3],
                    "b":[5, 6, 7]})

In [26]:
df1

,a,b
0,1,5
1,2,6
2,3,7
3,4,8


In [27]:
df2

,a,b
0,1,5
1,2,6
2,3,7


In [30]:
df1.append(df2.iloc[2],ignore_index=True)

,a,b
0,1,5
1,2,6
2,3,7
3,4,8
4,3,7


In [32]:
df2.drop(df2.index[2],inplace=True)

In [63]:
df2

,a,b
0,1,5
1,2,6
